In [2]:
%load_ext rpy2.ipython

In [3]:
library(dplyr)
library(data.table)
library(rjson)
library(stringr)
library(jsonlite)
library(httr)
library(purrr)
library(tidyverse)
library(readxl)
library(DT)
library(xlsx)
library(plotly)

NameError: ignored

In [ ]:
json2df = function(a){

# "a" é a URL
f_api <-   GET(a)

f_txt <- content(f_api, as="text", encoding="UTF-8")

f_json <- fromJSON(f_txt, flatten = FALSE)
Sys.sleep(1)

f_df <-as.data.frame(f_json[[1]])

}

In [ ]:
bind_json2df = function(a){ map(a,json2df)}

In [ ]:
relatorios = function(a){map_dfr(bind_json2df(a), bind_rows)}

In [ ]:
entes<- json2df("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/entes")

datatable(entes)

In [ ]:
asssembleia_ceara <- json2df( "http://apidatalake.tesouro.gov.br/ords/siconfi/tt/extrato_entregas?id_ente=23&an_referencia=2019")

datatable(asssembleia_ceara)

In [ ]:
arq<- c()


In [ ]:
entes_knit<-entes%>%filter(ente %in% c("Florianópolis","Blumenau","Joinville"))

In [ ]:
for (i in entes_knit$cod_ibge){

files_json <-
  str_replace("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/extrato_entregas?id_ente=XXXXXX&an_referencia=2019","XXXXXX",toString(i))

arq<- c(arq, files_json)
}

In [ ]:
arq_ente<- data.frame ( "ente" =  entes_knit$ente, "cod_ibge" = entes_knit$cod_ibge, "url_extrato"=arq  )

In [ ]:
# criar variáveis do loop
extratos <- c()
extratos_urls<-c()
status_api<- c()
status_cod_ibge<- c()

# loop em arq para baixar o extrato individual de cada ente
for (i in arq){
# acessa o API e baixa o arquivo JSON
ext_api<-   GET(i)

ext_txt <- content(ext_api, as="text", encoding="UTF-8")

ext_json <- fromJSON(ext_txt, flatten = FALSE)

ext  <-as.data.frame(ext_json[["items"]])

# juntar os extratos
extratos<- rbind(ext, extratos)

# verificar status da consulta
status_api<- c(status_code(ext_api),status_api )

# verificar se acessou corretamente a API
status_cod_ibge<- c(status_cod_ibge, ext_json[["items"]][["cod_ibge"]] )
}

In [ ]:
# criar variável "entregue" com situação "pendente" ou "entregue"
arq_ente<- arq_ente%>%
  mutate (entregue = if_else((entes_knit$cod_ibge %in% (unique(extratos$cod_ibge)))== FALSE, "pendente", "entregue"))

arq_ente%>%
  group_by(entregue)%>%
  dplyr::summarise(quantidade = n ())

In [ ]:
#filtrar apenas extratos pendentes
arq_ente_pendente <- arq_ente%>%
  filter (entregue == "pendente")


# loop em rq_ente_pendente para baixar os extratos pendente
for (i in arq_ente_pendente$url_extrato){
# acessa o API e baixa o arquivo JSON
ext_api<-   GET(i)


ext_txt <- content(ext_api, as="text", encoding="UTF-8")

ext_json <- fromJSON(ext_txt, flatten = FALSE)


ext <-as.data.frame(ext_json[["items"]])

extratos<- rbind(ext, extratos)

status_api<- c(status_code(ext_api),status_api )

status_cod_ibge<- c(status_cod_ibge, ext_json[["items"]][["cod_ibge"]] )


}

arq_ente%>%
  group_by(entregue)%>%
  dplyr::summarise(quantidade = n ())

In [ ]:
# tornar amigavel a df extratos
extratos<- extratos%>%
  mutate (data = as.Date (data_status))

# adicionar variável "tipo" (nome reduzido do entregavel)
extratos<- extratos%>%
  mutate (tipo =case_when(
 entregavel == "Balanço Anual (DCA)" ~ "DCA",
 entregavel == "MSC Agregada" ~ "MSCC",
 entregavel == "MSC Encerramento" ~ "MSCE",
 entregavel == "Relatório de Gestão Fiscal" ~ "RGF",
 entregavel == "Relatório de Gestão Fiscal Simplificado" ~ "RGF Simplificado",
 entregavel == "Relatório Resumido de Execução Orçamentária" ~ "RREO",
 entregavel == "Relatório Resumido de Execução Orçamentária Simplificado" ~ "RREO Simplificado"
))


# adicionar variável "poder" (L/legislativo, E/executivo, J/Judiciário...)
extratos<-extratos%>%
  mutate (poder = case_when(
    str_detect(instituicao,'Assembleia Legislativa') ~ "L",
    str_detect(instituicao,'Câmara de Vereadores') ~ "L",
    str_detect(instituicao,'Senado Federal') ~ "L",
    str_detect(instituicao,'Câmara dos Deputados') ~ "L",
    str_detect(instituicao,'Câmara Legislativa') ~ "L",
    str_detect(instituicao,'Prefeitura Municipal') ~ "E",
    str_detect(instituicao,'Governo do') ~ "E",
    str_detect(instituicao,'Governo Federal') ~ "E",
    str_detect(instituicao,'Tribunal') ~ "J",
    str_detect(instituicao,'Conselho da Justiça Federal') ~ "J",
    str_detect(instituicao,'Conselho Nacional de Justiça') ~ "J",
    str_detect(instituicao,'Ministério Público') ~ "M",
    str_detect(instituicao,'Defensoria') ~ "D",
    str_detect(instituicao,'CONSÓRCIO') ~ "C",
    str_detect(instituicao,'CONSORCIO') ~ "C",
    str_detect(instituicao,'Consórcio') ~ "C",
    TRUE ~ "O"
  ))

# mesclar df entes e extratos para carregar na df extratos as variáveis (uf, ente, regiao, esfera, capital)
entes_merge<-entes%>%
  select( ente, uf, regiao, esfera, capital, cod_ibge)

extratos<- merge(extratos, entes_merge, by = "cod_ibge")

# cria variável ente_uf para diferenciar cidades com mesmo nome
extratos<-extratos%>%
  mutate(ente_uf = paste0(ente,"-",uf))

In [ ]:
extratos_msc<-extratos%>%
  filter(tipo %in% c("MSCC"))%>%
  group_by(ente,tipo, cod_ibge, regiao, uf, esfera, poder, exercicio,populacao, ente_uf)%>%
  # max(periodo) para capturar apenas a matriz mais recente
  summarise(recente = max(periodo))

# criar link para API
extratos_msc<-extratos_msc%>%
  mutate(classe_1 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_patrimonial?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=1&id_tv=ending_balance") )%>%
  mutate(classe_2 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_patrimonial?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=2&id_tv=ending_balance" ))%>%
  mutate(classe_3 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_patrimonial?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=3&id_tv=ending_balance") )%>%
  mutate(classe_4 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_patrimonial?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=4&id_tv=ending_balance" ))%>%
  mutate(classe_5 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_orcamentaria?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=5&id_tv=ending_balance" ))%>%
  mutate(classe_6 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_orcamentaria?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=6&id_tv=ending_balance" ))%>%
  mutate(classe_7 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_controle?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=7&id_tv=ending_balance" ))%>%
  mutate(classe_8 = paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_controle?id_ente=",cod_ibge,"&an_referencia=",exercicio,"&me_referencia=",recente,"&co_tipo_matriz=MSCC&classe_conta=8&id_tv=ending_balance" ))

In [ ]:
# criar df extratos_relatorios (excluir MSCC e MSCE)
extratos_relatorios<-extratos%>%
  filter(tipo != ("MSCC"),tipo != ("MSCE") )

# criar link para API
extratos_relatorios<-extratos_relatorios%>%
 mutate (url_api =case_when(
 tipo == "DCA" ~ paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/dca?an_exercicio=",exercicio,"&id_ente=",cod_ibge),
 tipo == "MSCA" ~ "MSCA",
 tipo == "MSCE" ~ "MSCE",
 tipo == "RGF" ~ paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/rgf?an_exercicio=",exercicio,"&in_periodicidade=",periodicidade,"&nr_periodo=",periodo,"&co_tipo_demonstrativo=RGF&co_poder=",poder,"&id_ente=",cod_ibge),
 tipo == "RGF Simplificado" ~ paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/rgf?an_exercicio=",exercicio,"&in_periodicidade=",periodicidade,"&nr_periodo=",periodo,"&co_tipo_demonstrativo=RGF%20Simplificado&co_poder=",poder,"&id_ente=",cod_ibge),
 tipo == "RREO" ~ paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/rreo?an_exercicio=",exercicio,"&nr_periodo=",periodo,"&co_tipo_demonstrativo=RREO&id_ente=",cod_ibge),
 tipo == "RREO Simplificado" ~ paste0("http://apidatalake.tesouro.gov.br/ords/siconfi/tt/rreo?an_exercicio=",exercicio,"&nr_periodo=",periodo,"&co_tipo_demonstrativo=RREO%20Simplificado&id_ente=",cod_ibge)

) )

In [ ]:
# selecionar o RGF (tipo == "RGF") do segundo quadrimenestre (periodo ==2) das câmaras municipais (poder == "L")
rgf_url<-extratos_relatorios%>%
  filter(  tipo == "RGF", poder =="L", periodo == 2)

# obter a RGF a partir da URL
rgf <-relatorios(rgf_url$url_api)

# filtrar apenas a despesa total com pessoal nos últimos 12 meses
rgf_pessoal<-rgf%>%
  filter ( coluna == "TOTAL (ÚLTIMOS 12 MESES) (a)",
           cod_conta == "DespesaComPessoalLiquida")%>%
  mutate(despesa = valor/1000000)%>%
  arrange(despesa)


# preparar df merge_mun que será mesclada à df rgf_pessoal
# o objetivo é agregar à df rgf_pessoal as variáveis: regiao, ente_uf, instituicao e ente.
merge_mun<-extratos%>%
  filter(tipo == "RGF", periodo == 2, esfera == "M")%>%
  select(regiao, ente_uf, instituicao, ente)

rgf_pessoal<- merge(rgf_pessoal, merge_mun, by = "instituicao")

# plotar o gráfico
 gasto<- ggplot()+
    geom_col(data = rgf_pessoal, aes( x=reorder (instituicao, despesa), y=despesa, fill= instituicao))+
   coord_flip()
 gasto <-ggplotly(gasto) %>%
  layout(showlegend = FALSE)%>%
  layout( title = "Despesa com pessoal entre set/2018 e ago/2019 (R$ milhões)",
          xaxis = list( title = "", showline = FALSE, showgrid = FALSE),
          yaxis = list ( title = "", showline = FALSE, showgrid = FALSE ))

(gasto)

 datatable(rgf_pessoal%>%
             select(instituicao, valor))

In [ ]:
# filtrar apenas Florianópolis
msc_fln_url<-extratos_msc%>%
  filter( cod_ibge == 4205407)

# criar df com a url do api de cada classe contábil (colunas 12 a 19 da df com os extratos das MSC)
classes_fln<- map_dfr(msc_fln_url[12:19], cbind.data.frame)

# renomear
names(classes_fln)[1] <- "url_api"

# aplicar a função
msc_fln<-relatorios((classes_fln$url_api))

# filtrar classe contábil 3 e agragar por conta contábil e natureza da conta (Débito ou Crédito)
msc_fln_classe_3<-msc_fln%>%filter(classe_conta == 3)%>%
  group_by( conta_contabil, natureza_conta)%>%
  summarise(saldo = (sum(valor)))

datatable(msc_fln_classe_3)

In [ ]:
extratos <- read_excel("extratos2.xlsx")

extratos%>%
  group_by(entregavel)%>%
  dplyr::summarise(quantidade = n ())